In [28]:
import os
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras import layers
from tensorflow.keras import models 
from tensorflow import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [2]:
os.listdir()

['VGG16_from_pretrain.ipynb', 'VGG16_from_scratch.ipynb']

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'c:\\Users\\dhruv\\Documents\\DA-IICT\\sem-1\\FOML\\DAIICT_FOML_Project\\Alzheimers_Disease_Detection'

#### Load the Dataset

In [ ]:
trainDataGen = ImageDataGenerator(
    rescale= 1./255,
    zoom_range= 0.2,
    horizontal_flip= True,
    validation_split= 0.2  
)

validationDataGen = ImageDataGenerator(
    rescale= 1./255,
    validation_split=0.2
)

testDataGen = ImageDataGenerator(
    rescale= 1./255 
)

print("geting the train data")

trainData = trainDataGen.flow_from_directory(
    "Data",
    subset="training",
    class_mode="categorical",
    target_size=(224,224)
)

print("geting the test data")

testData = testDataGen.flow_from_directory(
    "Test_Data",
    class_mode="categorical",
    target_size=(224,224)
)

print("geting the validation data")

validData = validationDataGen.flow_from_directory(
    "Data",
    subset="validation",
    class_mode="categorical",
    target_size=(224,224)
)

print("Data collected successfully")

geting the train data
Found 1581 images belonging to 4 classes.
geting the test data
Found 410 images belonging to 4 classes.
geting the validation data
Found 395 images belonging to 4 classes.
Data collected successfully


In [6]:
trainData.classes

array([0, 0, 0, ..., 3, 3, 3])

In [29]:
trainData.class_indices

{'Mild Dementia': 0,
 'Moderate Dementia': 1,
 'Non Demented': 2,
 'Very mild Dementia': 3}

In [30]:
num_class  = trainData.num_classes
num_class

4

In [31]:
label_to_class = {
0 : 'Mild Dementia',
1 : 'Moderate Dementia',
2 : 'Non Demented',
3 : 'Very mild Dementia'
} 

In [32]:
label_to_class

{0: 'Mild Dementia',
 1: 'Moderate Dementia',
 2: 'Non Demented',
 3: 'Very mild Dementia'}

In [33]:
input_shape = (224,224,3)
W = 224
H = 224

In [34]:
model = models.Sequential()
model 

In [35]:
## Block - 1
model.add(layers.Conv2D(64, (3,3), activation="relu", padding="same",input_shape = input_shape))
model.add(layers.Conv2D(64, (3,3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D((2,2), strides=(2,2)) )

## Block - 2
model.add(layers.Conv2D(128, (3,3), activation="relu", padding="same",))
model.add(layers.Conv2D(128, (3,3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D((2,2), strides=(2,2)) )

## Block - 3
model.add(layers.Conv2D(256, (3,3), activation="relu", padding="same",))
model.add(layers.Conv2D(256, (3,3), activation="relu", padding="same"))
model.add(layers.Conv2D(256, (3,3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D((2,2), strides=(2,2)) )

## Block - 4
model.add(layers.Conv2D(512, (3,3), activation="relu", padding="same",))
model.add(layers.Conv2D(512, (3,3), activation="relu", padding="same"))
model.add(layers.Conv2D(512, (3,3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D((2,2), strides=(2,2)) )

## Block - 5
model.add(layers.Conv2D(512, (3,3), activation="relu", padding="same",))
model.add(layers.Conv2D(512, (3,3), activation="relu", padding="same"))
model.add(layers.Conv2D(512, (3,3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D((2,2), strides=(2,2)) )

## Add Fully Connected Layers
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_class,activation="softmax"))


In [36]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      1792      
                                                                 
 conv2d_2 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_4 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                  

In [37]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      1792      
                                                                 
 conv2d_2 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_4 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                  

In [39]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [40]:
early_stopping = EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True)
model_check_point = ModelCheckpoint("Model/best_vgg16_from_scratch_model.h5", save_best_only=True, monitor ="val_loss")


In [44]:
history = model.fit(
    trainData,
    steps_per_epoch = trainData.samples,
    validation_data = validData,
    validation_steps = validData.samples // validData.batch_size,
    epochs = 20,
    callbacks = [early_stopping, model_check_point]
)

history

Epoch 1/20
1581/1581 [==============================] - 876s 545ms/step - loss: 1.3889 - accuracy: 0.2441 - val_loss: 1.3863 - val_accuracy: 0.2552


In [43]:
history.history

{'loss': [1.4785997867584229],
 'accuracy': [0.25553447008132935],
 'val_loss': [1.388003945350647],
 'val_accuracy': [0.2526041567325592]}